In [8]:
from flask import Flask, render_template, request
import pandas as pd
import portfolio_analytics.data
import os
from portfolio_analytics.classes.market_data import MarketData
from portfolio_analytics.classes.portfolio_decomposer import PortfolioDecomposer

In [9]:
app = Flask(__name__)

In [10]:
# Paths for portfolio_analytics data
DB_PATH = "portfolio_analytics/data/stocks.db"
META_PATH = "portfolio_analytics/data/etf_metadata.json"

In [11]:
# Initialize MarketData
market_data = MarketData(db_name=DB_PATH, meta_file=META_PATH)

UPLOAD_FOLDER = "uploads"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

In [12]:
@app.route("/")
def index():
    return render_template("index.html")

@app.route("/upload", methods=["POST"])
def upload_file():
    if "file" not in request.files:
        return "No file part"

    file = request.files["file"]
    if file.filename == "":
        return "No selected file"

    # Save the file
    filepath = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(filepath)

    # Read the CSV/XLS file
    df = pd.read_csv(filepath) if file.filename.endswith(".csv") else pd.read_excel(filepath)

    # Process Portfolio
    portfolio_analyzer = PortfolioDecomposer(port=df, market_data=market_data)
    decomposed_stocks = portfolio_analyzer.decompose_stocks()
    decomposed_sectors = portfolio_analyzer.decompose_sectors()
    #print(decomposed_stocks)

    return render_template(
        "results.html",
        stocks=decomposed_stocks.to_html(classes="table"),
        sectors=decomposed_sectors.to_html(classes="table"),
    )

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\HP\Desktop\flask-app\venv\Lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
